### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt5
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
from matplotlib import interactive
interactive(True)
import os
import matplotlib.patches as plm

Failed loading module Labbrick:  [WinError 126] The specified module could not be found
Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


# Спектроскопия

In [2]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'GEN1')

In [3]:
current_src1 = Yokogawa_GS210(address = 'GPIB0::1::INSTR')
current_src2 = Yokogawa_GS210(address = 'GPIB0::2::INSTR')

In [4]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Резонаторная спектроскопия

### Одномерный скан при фиксированном токе 

In [18]:
pna.set_nop(3501)
pna.set_xlim(6e9,8.8e9)
pna.set_power(-50)
pna.set_bandwidth(10)
s21=pna.get_tracedata()
fr=pna.get_freqpoints()
plt.plot(fr[:len(fr)-1],np.unwrap(np.diff(s21[1])))

In [12]:
#Резонансные частоты при curr=1e-5 (Последние два резонатора - тестовые)
#res_fr=np.asarray([6.39e9,6.47e9,6.58e9,6.60e9,6.68e9,6.76e9,7.29e9,7.95e9]) 
res_fr=np.asarray([6.39e9,6.58e9,6.60e9,6.68e9])#,6.76e9,7.29e9,7.95e9])
qubit_params = {1: {'Fr': 6.676e9,'F01_max': 8.172e9, 'Iw':-2.5e-5}}
                #2: {'Fr': 6.58e9,'F01_max': 7.889e9, 'Anh':50e6, 'Iw':1e-5},
               #3: {'Fr': 6.68e9,'F01_max': 8.169e9, 'Anh':230e6,'Iw':8e-6 }
curr_range=np.asarray([(-4.5e-5,-0.5e-5),(-3e-5,0),(-35e-5,10e-5),(-5e-5,0)])
current_src1.set_current(qubit_params[1]['Iw'])

True

In [31]:
lo1.set_status(0)
pna.set_nop(301)
pna.set_bandwidth(200)
pna.set_power(-50)
current_max=5e-5
current_nop=201
currents=np.linspace(-5e-5,0,current_nop)
for res in res_fr:
    pna.set_xlim(res-25e6,res+25e6)
    result = sweep.sweep(pna, (currents, current_src1.set_current, 'Coil current'), filename='anticrossings Rev_{0}_{1}'.format("%.2f" % (res/1e9),pna.get_power()))

A Jupyter Widget

Started at:  Mar 04 2018 18:14:26


RuntimeError: You must first define an image, e.g., with imshow

## Кубитная спектроскопия

In [36]:
dfr=0.01e9
nop=301
resolution_bw=10
tone1_power=-55
cur_nop=101
tone2_power=5

lo1.set_status(1)
lo1.set_power(tone2_power)
freq=np.linspace(7e9,9.5e9,301)
for i in range(3,len(res_fr)):
    tone1_freq_min, tone1_freq_max = res_fr[i]-dfr, res_fr[i]+dfr
    currents=np.linspace(-5e-5,0,current_nop)
    sweep.sweep(pna, (currents, curr,'current'), (freq, lo1.set_frequency, '2nd tone frequency'), filename='2nd tone_fr{0}_t1p{1}_t2p{2}'.format("%.2f" % (res_fr[i]/1e9),pna.get_power(),lo1.get_power()))


A Jupyter Widget

Started at:  Mar 04 2018 18:22:00

Elapsed time: 5 h 9 m 45.58 s


In [12]:
def curr(current):
    lo1.set_status(0)
    pna.set_average(0)
    pna.set_power(-50)
    pna.set_nop(nop)
    pna.set_xlim(tone1_freq_min, tone1_freq_max)
    pna.set_bandwidth(200)
    current_src1.set_current(current)
    S21   = pna.get_tracedata()
    freqs = pna.get_freqpoints()
    pna.set_bandwidth(resolution_bw)
    pna.set_power(tone1_power)
    pna.set_nop(1)
    pna.set_xlim(freqs[np.argmin(S21[0])]+0.25e6,freqs[np.argmin(S21[0])]+0.25e6)
    lo1.set_status(1)

In [30]:
curr(-2.7e-5)
powers=np.linspace(-10,10,21)
pna.set_power(-50)
freq=np.linspace(7e9,9.5e9,501)
sweep.sweep(pna, (powers,lo1.set_power,'2-nd tone power'), (freq, lo1.set_frequency, '2nd tone frequency'), filename='2nd tone_power_scan_curr_{0}_t1p{1}'.format("%.2f" % (current_src1.get_current()*1e5),pna.get_power()))


A Jupyter Widget

Started at:  Mar 04 2018 16:38:13

Elapsed time: 0 h 52 m 12.46 s


{'S-parameter': (('2-nd tone power', '2nd tone frequency'),
  (array([-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
            1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.]),
   array([  7.00000000e+09,   7.00500000e+09,   7.01000000e+09,
            7.01500000e+09,   7.02000000e+09,   7.02500000e+09,
            7.03000000e+09,   7.03500000e+09,   7.04000000e+09,
            7.04500000e+09,   7.05000000e+09,   7.05500000e+09,
            7.06000000e+09,   7.06500000e+09,   7.07000000e+09,
            7.07500000e+09,   7.08000000e+09,   7.08500000e+09,
            7.09000000e+09,   7.09500000e+09,   7.10000000e+09,
            7.10500000e+09,   7.11000000e+09,   7.11500000e+09,
            7.12000000e+09,   7.12500000e+09,   7.13000000e+09,
            7.13500000e+09,   7.14000000e+09,   7.14500000e+09,
            7.15000000e+09,   7.15500000e+09,   7.16000000e+09,
            7.16500000e+09,   7.17000000e+09,   7.17500000e+09,
            7.18000000e+

In [621]:
with open('D:\\qtlab\\ReiData\\data\\2018-03-01\\16-14-33-2nd tone\\S-parameter 2nd tone.pkl', 'rb') as f:
    data_new = pickle.load(f)
mean = np.mean(data_new[1]['S-parameter'][2], axis=1)
plt.pcolormesh(data_new[1]['S-parameter'][1][0],data_new[1]['S-parameter'][1][1], np.abs(data_new[1]['S-parameter'][2].T-mean), cmap='RdBu')
plt.savefig('D:\\qtlab\\ReiData\\data\\2018-03-01\\16-14-33-2nd tone\\2tone.jpg',format='jpg')

In [41]:

with open('D:\\qtlab\\ReiData\\data\\2018-03-04\\09-52-59-anticrossings Rev_6.68_-50.0\\S-parameter anticrossings Rev_6.68_-50.0.pkl', 'rb') as f:
    data_new = pickle.load(f)     
plt.pcolormesh(data_new[1]['S-parameter'][1][0],data_new[1]['S-parameter'][1][1], np.abs(data_new[1]['S-parameter'][2].T), cmap='RdBu')

# Импульсы

### Создаём девайсы

In [5]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


### Загружаем импульсные скрипты всякие

In [6]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

In [7]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

### AWGшки

In [29]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 125e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,5):
    awg_tek.set_amplitude(.2, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 2, 1, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(qubit_params[1]['F01_max'])
iq_ro.set_frequency(qubit_params[1]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [30]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())

### Калибровка миксеров

In [39]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

D:/qtlab/ReiData/data/calibrations//IQ-if1.2e+08-rf8.2e+09-sb--1.pkl
8297000000.0
[ 0.1  0.1] -12.3134803772
[ 0.105  0.1  ] -12.0859746933
[ 0.1    0.105] -12.269908905
[ 0.095  0.105] -12.5024557114
[ 0.09    0.1075] -12.7268266678
[ 0.09    0.1025] -12.7866973877
[ 0.085    0.10125] -13.0398731232
[ 0.075    0.10875] -13.4398527145
[ 0.0625    0.113125] -13.9457654953
[ 0.0575    0.106875] -14.3045902252
[ 0.04125    0.1065625] -15.2071247101
[ 0.01875    0.1184375] -16.259727478
[-0.014375    0.12703125] -17.9034976959
[-0.035625    0.12046875] -19.9011173248
[-0.0846875   0.12414062] -22.1487579346
[-0.1403125   0.14460937] -18.3273448944
[-0.210625    0.14171875] -14.9806671143
[-0.0634375   0.13070312] -20.5513343811
[-0.0078125   0.11023437] -18.5115966797
[-0.0409375   0.11882812] -20.5751132965
[-0.0621875   0.11226562] -22.9339065552
[-0.0615625   0.10304687] -24.1654319763
[-0.1053125   0.10835937] -24.7547931671
[-0.1375    0.103125] -22.6090259552
[-0.0821875   0.08726562

dc: -9.07e-02+6.91e-02j	I:  5.29e-01+5.15e-01j	Q: 4.55e-01+5.09e-01j	B: -39.39 G: -0.67, C:0.00
[-68.37761688 -67.33342743  -0.67074686 -48.90256119 -45.82244492
 -60.67054749 -70.70664978]
dc: -9.09e-02+6.99e-02j	I:  5.36e-01+5.10e-01j	Q: 4.40e-01+5.18e-01j	B: -35.02 G: -0.68, C:0.00
[-70.09578705 -68.7826004   -0.67929888 -50.82753372 -37.89163971
 -59.37530899 -68.64406586]
dc: -9.16e-02+6.91e-02j	I:  5.28e-01+5.16e-01j	Q: 4.53e-01+5.04e-01j	B: -40.07 G: -0.70, C:0.00
[-66.54749298 -67.70582581  -0.70184499 -50.49633789 -46.6138916
 -59.65597153 -69.24192047]
dc: -8.98e-02+6.87e-02j	I:  5.41e-01+5.16e-01j	Q: 4.50e-01+5.06e-01j	B: -38.88 G: -0.66, C:0.00
[-67.03884125 -66.4984436   -0.65597528 -50.16281128 -44.34605789
 -60.79182816 -68.33696747]
dc: -9.05e-02+6.89e-02j	I:  5.37e-01+5.15e-01j	Q: 4.49e-01+5.07e-01j	B: -40.21 G: -0.68, C:0.00
[-67.57761383 -68.55558777  -0.67696649 -46.95503998 -50.02535629
 -60.22817612 -69.17195129]
dc: -9.05e-02+6.90e-02j	I:  5.38e-01+5.08e-01j	Q: 4

dc: -9.06e-02+6.93e-02j	I:  5.35e-01+5.14e-01j	Q: 4.53e-01+5.06e-01j	B: -45.96 G: -0.66, C:0.00
[-68.41493225 -68.6520462   -0.66374981 -54.22267532 -59.60902786
 -60.99474335 -67.88993073]
dc: -9.07e-02+6.92e-02j	I:  5.34e-01+5.14e-01j	Q: 4.52e-01+5.06e-01j	B: -44.11 G: -0.67, C:0.00
[-67.42212677 -68.37055206  -0.66608214 -50.08972931 -60.21699524
 -60.29347992 -69.00169373]
dc: -9.09e-02+6.91e-02j	I:  5.34e-01+5.14e-01j	Q: 4.52e-01+5.07e-01j	B: -45.12 G: -0.67, C:0.00
[-67.18656158 -66.33673096  -0.66608214 -51.23025513 -65.10406494
 -60.43342209 -68.57720184]
dc: -9.08e-02+6.92e-02j	I:  5.34e-01+5.14e-01j	Q: 4.52e-01+5.06e-01j	B: -47.61 G: -0.67, C:0.00
[-67.76187134 -68.56414032  -0.67385668 -56.46796036 -64.45567322
 -60.75139999 -68.79566193]
dc: -9.08e-02+6.91e-02j	I:  5.35e-01+5.14e-01j	Q: 4.51e-01+5.07e-01j	B: -46.16 G: -0.67, C:0.00
[-68.25788879 -66.08328247  -0.66608214 -55.75114822 -59.05003738
 -60.34090805 -69.59178162]
dc: -9.08e-02+6.90e-02j	I:  5.35e-01+5.14e-01j	Q: 

dc: -9.07e-02+6.91e-02j	I:  5.35e-01+5.14e-01j	Q: 4.52e-01+5.06e-01j	B: -46.86 G: -0.68, C:0.00
[-67.51317596 -66.29949951  -0.67549628 -55.35239792 -62.55332565
 -61.63700485 -69.02820587]
D:/qtlab/ReiData/data/calibrations/
D:/qtlab/ReiData/data/calibrations//IQ-if7.5e+07-rf6.7e+09-sb--1.pkl
6751000000.0
[ 0.1  0.1] -51.40599823
[ 0.105  0.1  ] -52.2148780823
[ 0.1    0.105] -50.8755226135
[ 0.105  0.095] -52.6448631287
[ 0.1075  0.09  ] -53.6005935669
[ 0.1125  0.09  ] -54.8319358826
[ 0.11875  0.085  ] -57.0238952637
[ 0.12125  0.075  ] -59.7405967712
[ 0.129375  0.0625  ] -63.8075828552
[ 0.140625  0.0575  ] -65.8686828613
[ 0.1571875  0.04125  ] -55.7526283264
[ 0.15125  0.035  ] -54.8157615662
[ 0.126875  0.0725  ] -63.134513855
[ 0.143125  0.0475  ] -59.1840324402
[ 0.1309375  0.06625  ] -67.9460601807
[ 0.1421875  0.06125  ] -68.1016769409
[ 0.14859375  0.060625  ] -62.5710372925
[ 0.1325  0.07  ] -69.8663330078
[ 0.1284375  0.07625  ] -63.2738761902
[ 0.14375  0.065  ] -67.32

dc:  1.47e-01+6.28e-02j	I:  1.32e-01+6.76e-01j	Q: 6.75e-01+5.06e-01j	B: -44.29 G: -27.29, C:0.00
[-80.09480286 -78.13818359 -27.2886982  -68.06050873 -45.66026306
 -74.03322601 -82.52542877]
dc:  1.46e-01+6.39e-02j	I:  1.91e-01+6.50e-01j	Q: 6.40e-01+5.00e-01j	B: -49.43 G: -27.33, C:0.00
[-80.93833923 -77.23867035 -27.32523727 -69.58509064 -51.81303024
 -75.59823608 -81.55516815]
dc:  1.49e-01+6.84e-02j	I:  1.02e-01+6.76e-01j	Q: 6.38e-01+5.19e-01j	B: -44.26 G: -27.43, C:0.00
[-80.69810486 -77.53876495 -27.42630768 -64.29063416 -46.03654861
 -73.86995697 -80.45040894]
dc:  1.43e-01+6.37e-02j	I:  2.84e-01+6.12e-01j	Q: 6.03e-01+4.88e-01j	B: -49.96 G: -27.31, C:0.00
[-80.46720123 -78.47015381 -27.31046677 -74.11842346 -51.97473907
 -75.63166809 -81.73709106]
dc:  1.49e-01+6.89e-02j	I:  1.40e-01+6.83e-01j	Q: 6.42e-01+4.63e-01j	B: -44.66 G: -27.51, C:0.00
[-82.21647644 -78.10863495 -27.51260376 -62.80025482 -46.66473007
 -73.9865799  -82.05351257]
dc:  1.43e-01+6.38e-02j	I:  2.61e-01+6.14e-01

dc:  1.43e-01+6.23e-02j	I:  2.39e-01+6.62e-01j	Q: 6.06e-01+4.91e-01j	B: -58.06 G: -27.22, C:0.00
[-82.37541199 -77.75411987 -27.22183609 -72.39558411 -65.34909821
 -75.16053009 -80.96392822]
dc:  1.42e-01+6.41e-02j	I:  2.59e-01+6.49e-01j	Q: 6.07e-01+4.86e-01j	B: -55.93 G: -27.21, C:0.00
[-82.01433563 -78.80989838 -27.21483994 -73.80200195 -60.47386169
 -75.5313797  -81.38490295]
dc:  1.44e-01+6.32e-02j	I:  2.30e-01+6.64e-01j	Q: 6.07e-01+4.90e-01j	B: -58.23 G: -27.25, C:0.00
[-82.32376099 -78.33798981 -27.24671555 -71.83660126 -65.47737885
 -76.64391327 -80.62999725]
dc:  1.43e-01+6.42e-02j	I:  2.49e-01+6.58e-01j	Q: 6.03e-01+4.90e-01j	B: -57.68 G: -27.21, C:0.00
[-82.99081421 -78.72593689 -27.21017456 -75.48363495 -63.29446411
 -75.14809418 -80.68209076]
dc:  1.43e-01+6.33e-02j	I:  2.35e-01+6.60e-01j	Q: 6.08e-01+4.89e-01j	B: -58.25 G: -27.25, C:0.00
[-82.25067902 -78.28356171 -27.24671555 -72.84495544 -65.35842896
 -74.83322906 -82.45857239]
dc:  1.43e-01+6.45e-02j	I:  2.39e-01+6.66e-01

{'I': (0.24764264232158317+0.65890104903789948j),
 'Q': (0.61423155430803233+0.48877158473967863j),
 'dc': (0.14143137026361272+0.065077569347558845j),
 'num_sidebands': 7,
 'score': -38.572228763045096}

### Длительность и амплитуда считывающих импульсов

In [40]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*3/4
    
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [66]:
ro_adc_length = 1e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude = 0.24
ro_dac_length = 0.5e-6
adc.set_nums(20000)
#adc.set_software_nums_multi(1)
set_adc_nop(ro_adc_length)

1024


In [63]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.plot(np.mean(np.real(adc.measure()['Voltage']), axis=0))

In [ ]:
plt.pcolormesh(np.real(adc.measure()['Voltage']));
plt.colorbar()

In [42]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [65]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 51), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

A Jupyter Widget

Started at:  Mar 06 2018 09:32:56


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 3 m 41.22 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


### Осцилляции Раби

In [67]:
ex_amplitude = 0.5
pause_length = 16e-9
lengths = np.linspace(0e-9, 0.2e-6, 41)
readout_begin = np.max(lengths)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

A Jupyter Widget

Started at:  Mar 06 2018 09:37:12


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 2 m 33.32 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

### Осцилляции Рамзея

In [60]:
ex_amplitude = 0.5
pi2_pulse = 10e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 2000e-9, 301)
target_freq_offset = 7e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
#save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Mar 05 2018 21:37:22


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 16 m 47.74 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [61]:
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.5
pi_pulse = 22e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 10000e-9, 201)
readout_begin = np.max(delays)+pi_pulse
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-ro_dac_length),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Mar 06 2018 09:47:32


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


In [ ]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

### Rabi amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.,1.0,51)
pause_length = 32e-9
length = 32e-9
sigma = 8e-9
num_pulses = 4
readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', pi_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, readout_begin-length)]*num_pulses+\ 
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

### gaussian $\pi/2$ pulse amplitudes

In [ ]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x = 0.925
length = 32e-9
pause_length = 8e-9
sigma = 8e-9
amp_x_hd = 0.925
length_hd = 32e-9
sigma_hd = 8e-9
pause_length_hd = 8e-9
alpha_hd = -0.92e-9

### Ramsey with gauss pulses

In [ ]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), 0), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

# Optimizing readout pulse settings

In [ ]:
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma), pg.p(None, pause_length)]*2
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc.set_software_nums_multi(20)
adc_sz.repeat_samples = 1

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_amplitude, ro_dac_length):
        self.ro_amplitude = ro_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0, 0.05e-6, 6)
ro_dac_amplitudes = np.linspace(0.8,1.0, 5)

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_amplitude, ro_dac_length)
solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)
ro_dac_length = solution[0]
ro_dac_amplitude = solution[1]
#
#sweep.sweep(adc_sz, (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
#                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'))

### |0> & |1> readout calibration

In [ ]:
ro_dac_length = 50e-9
ro_dac_amplitude = 0.13
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]*2
#ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, 1.0)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc_sz.save_last_samples = True
adc_sz.repeat_samples = 1
adc.set_software_nums_multi(20)
print(adc_sz.measure())
#sz_filter = adc_sz.calibrate()
#print('ROC AUC:', adc_sz.calib_roc_auc, 'Fidelity:', adc_sz.calib_fidelity)
#print('ROC AUC binary:', adc_sz.calib_roc_auc_binary, 'Fidelity binary:', adc_sz.calib_fidelity_binary) 

In [ ]:
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[0,:,:]), vmin=15, vmax=25); plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[1,:,:]), vmin=15, vmax=25); plt.colorbar()

In [ ]:
plt.hist(adc_sz.calib_pred.T, bins=20)

In [ ]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
#plt.figure()
#plt.hist(adc_sz.calib_pred.T, bins=20)
plt.figure()
#plt.hist(np.real(adc_sz.calib_usl_pred.T), bins=20)
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.imag(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

### Tomography

In [ ]:
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq)

In [ ]:
## tomography of |0> state
adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*2)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, 1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

# Randomized benchmarking

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomo)

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = {'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
                      'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
                      '-X':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
                      '-Y':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
                      'I':{'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
bench.sequence_length = 10
bench.random_sequence_num = 6
bench.prepare_random_interleaving_sequences()
#bench.generate_interleaver_sequence_from_names()

In [ ]:
seq_lengths = np.arange(101)
sweep.sweep(bench, (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number'))

In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))